<a href="https://colab.research.google.com/github/somendrew/LLMs/blob/main/Tool_Calling_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps
* Define Tool
* Describe Tool
* User Query
* Call LLM
* Check if tool is called
* Execute the function
* Send result back to model

 ****Tool Binding****

Tool Binding is a step where you register tools with a LLM so that:
* The LLM knows **what tools are available**
* It knows **what each tool does** (via discription)
* It knows **what input format** to use (via schema)

In [1]:
import os

from google.colab import userdata
api_key = userdata.get('api_key')

In [2]:
from openai import OpenAI
import langchain

In [3]:
# Define tool
from langchain.tools import tool

@tool
def multiply(a: int,b: int ) -> int :
  "Multiply two integers and return the Result:"
  return a*b


In [4]:
#User Query

query = 'what is the multiplication of 3 and 6'
messages = [{'role':'user', 'content':query}]

In [5]:
messages

[{'role': 'user', 'content': 'what is the multiplication of 3 and 6'}]

In [6]:
from langchain_core.utils.function_calling import convert_to_openai_function

client = OpenAI(api_key=api_key)

# -----------------------------
# 2. Describe tool to the LLM
# -----------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First number"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"]
            }
        }
    }
]

response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = messages,
    tools = tools,
    tool_choice ='auto'
)

In [7]:
import json
# Check if tool is called
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function
arguments = json.loads(tool_call.function.arguments)
tool_call

ChatCompletionMessageFunctionToolCall(id='call_jmjJ1ymFs4Rf3RjWhwPQ3pfm', function=Function(arguments='{"a":3,"b":6}', name='multiply'), type='function')

In [8]:
arguments

{'a': 3, 'b': 6}

In [9]:
response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_jmjJ1ymFs4Rf3RjWhwPQ3pfm', function=Function(arguments='{"a":3,"b":6}', name='multiply'), type='function')])

In [10]:
# 1. Ensure we are accessing the tool call object correctly
tool_call = response.choices[0].message.tool_calls[0]
function_args = json.loads(tool_call.function.arguments)
function_name = tool_call.function.name

result = None

# # 2. Execute the local function mapping
# if function_name == "multiply":
#     # Ensure 'multiply' is the tool object and 'func' is the callable
#     result = multiply.func(a=function_args.get('a'), b=function_args.get('b'))
# else:
#     print(f"Warning: Unexpected function name: {function_name}")

# 3. Reconstruct messages (Crucial: tool_calls must be in the assistant message)
messages_for_final_call = [
    {"role": "user", "content": query},
    response.choices[0].message, # Keep the original assistant message object
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": str(result)
    }
]

# 4. Final API Call
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages_for_final_call
)

print(final_response.choices[0].message.content)

The multiplication of 3 and 6 is 18.


### ToolKit


In [11]:
from langchain_core.tools import tool

@tool
def sum(a: int,b: int)-> int:
  "Add two numbers"
  return a+b

@tool
def multiply(a: int,b: int)-> int:
  "Multiply two numbers"
  return a*b


class MathToolKit:
  def get_tools(self):
    return [sum,multiply]

toolkit = MathToolKit()
tools = toolkit.get_tools()
for tool in tools:
  print(tool.name,"->",tool.description)

sum -> Add two numbers
multiply -> Multiply two numbers
